# **Shopee Code League - Order Brushing**

This notebook is for Shopee Code League Competition, Order Brushing, to detecting abnormal user behaviour. The competition details can be obtained from [here](https://www.kaggle.com/c/order-brushing-shopee-code-league/overview).

# **Import libraries and read dataset**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from datetime import datetime
from datetime import timedelta
from google.colab import files

In [2]:
from tensorflow.python.client import device_lib
check_device = device_lib.list_local_devices()
check_device

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 921571683999915116, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3596168475916917880
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17786568438488224976
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11150726272
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5537338114247512700
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [3]:
# order_brush_order.csv
d = pd.read_csv('order_brush_order.csv', index_col=0).sort_values(by=['event_time','shopid','userid']).reset_index()
len(d.shopid.unique())

18770

In [4]:
d

,orderid,shopid,userid,event_time
0,31075200506751,6042309,97707522,2019-12-27 00:00:00
1,31075200506753,8715449,97707522,2019-12-27 00:00:00
2,31075200506752,104804492,97707522,2019-12-27 00:00:00
3,31075201870570,190969466,170182475,2019-12-27 00:00:02
4,31075205798264,2859407,12532131,2019-12-27 00:00:05
...,...,...,...,...
222745,31507183252446,149254894,193333760,2019-12-31 23:59:43
222746,31507187390691,147941492,40258063,2019-12-31 23:59:47
222747,31507191066627,154074176,2338306,2019-12-31 23:59:51
222748,31507191066628,187123853,2338306,2019-12-31 23:59:51


In [5]:
# order_brush_order.csv
data = pd.read_csv('test_brush.csv', index_col=0).sort_values(by=['event_time','shopid','userid']).reset_index()
data

,orderid,shopid,userid,event_time
0,31388857391599,91242967,112494421,2019-12-30 15:07:37
1,31388858606904,85669731,112494421,2019-12-30 15:07:38
2,31388860144322,85669731,112494421,2019-12-30 15:07:41
3,31388862051153,85669731,112494421,2019-12-30 15:07:42
4,31388864143841,68986766,65129646,2019-12-30 15:07:45
5,31388864143842,68986766,86829555,2019-12-30 15:07:45
6,31388864143855,68986766,86829555,2019-12-30 15:07:45
7,31388864143886,68986766,86829555,2019-12-30 15:07:45
8,31388864143840,85669731,10209247,2019-12-30 15:07:45
9,31388864656540,85669731,112494421,2019-12-30 15:07:45


In [6]:
date = data['event_time']
date_time = []
for i in date:
  date_time.append(datetime.strptime(i,"%Y-%m-%d %H:%M:%S"))

In [7]:
data['date_time'] = date_time
data

,orderid,shopid,userid,event_time,date_time
0,31388857391599,91242967,112494421,2019-12-30 15:07:37,2019-12-30 15:07:37
1,31388858606904,85669731,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
2,31388860144322,85669731,112494421,2019-12-30 15:07:41,2019-12-30 15:07:41
3,31388862051153,85669731,112494421,2019-12-30 15:07:42,2019-12-30 15:07:42
4,31388864143841,68986766,65129646,2019-12-30 15:07:45,2019-12-30 15:07:45
5,31388864143842,68986766,86829555,2019-12-30 15:07:45,2019-12-30 15:07:45
6,31388864143855,68986766,86829555,2019-12-30 15:07:45,2019-12-30 15:07:45
7,31388864143886,68986766,86829555,2019-12-30 15:07:45,2019-12-30 15:07:45
8,31388864143840,85669731,10209247,2019-12-30 15:07:45,2019-12-30 15:07:45
9,31388864656540,85669731,112494421,2019-12-30 15:07:45,2019-12-30 15:07:45


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   orderid     17 non-null     int64         
 1   shopid      17 non-null     int64         
 2   userid      17 non-null     int64         
 3   event_time  17 non-null     object        
 4   date_time   17 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 808.0+ bytes


In [9]:
b = data['date_time'].loc[12]
a = data['date_time'].loc[1]
delt = b-a
delt.total_seconds()

10.0

In [10]:
check = data.loc[0:10].groupby('shopid')
check.first()

,orderid,userid,event_time,date_time
shopid,,,,
68986766,31388864143841,65129646,2019-12-30 15:07:45,2019-12-30 15:07:45
85669731,31388858606904,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
91242967,31388857391599,112494421,2019-12-30 15:07:37,2019-12-30 15:07:37


In [11]:
key = list(check.groups.keys())
key

[68986766, 85669731, 91242967]

In [12]:
key[1]

85669731

In [13]:
for k in key:
  print(k)

68986766
85669731
91242967


In [14]:
check_order = check.get_group(key[1])
check_order

,orderid,userid,event_time,date_time
1,31388858606904,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
2,31388860144322,112494421,2019-12-30 15:07:41,2019-12-30 15:07:41
3,31388862051153,112494421,2019-12-30 15:07:42,2019-12-30 15:07:42
8,31388864143840,10209247,2019-12-30 15:07:45,2019-12-30 15:07:45
9,31388864656540,112494421,2019-12-30 15:07:45,2019-12-30 15:07:45


In [15]:
check_order = check.get_group(key[1])
order = len(check_order.orderid)
buyers = len(check_order.userid.unique())
concentrate_rate = order/buyers
print(concentrate_rate)

2.5


In [16]:
charc = 'x'
haha = 'abc'
a_dict = {}
if charc in a_dict:
  a_dict[charc].append(haha)
else:
  a_dict[charc] = [haha]
print(a_dict)

{'x': ['abc']}


In [17]:
check = data.loc[1:13].groupby('shopid')

In [18]:
key = list(check.groups.keys())
key

[68986766, 85669731, 91242967]

In [19]:
check_order = check.get_group(68986766)
check_order

,orderid,shopid,userid,event_time,date_time
4,31388864143841,68986766,65129646,2019-12-30 15:07:45,2019-12-30 15:07:45
5,31388864143842,68986766,86829555,2019-12-30 15:07:45,2019-12-30 15:07:45
6,31388864143855,68986766,86829555,2019-12-30 15:07:45,2019-12-30 15:07:45
7,31388864143886,68986766,86829555,2019-12-30 15:07:45,2019-12-30 15:07:45
12,31388868340962,68986766,65129646,2019-12-30 15:07:48,2019-12-30 15:07:48
13,31388869521738,68986766,65129646,2019-12-30 15:07:49,2019-12-30 15:07:49


In [20]:
user_num = pd.DataFrame(check_order.groupby('userid')['userid'].count().reset_index(name='user_count'))
check_user = user_num[user_num['user_count'] >= 3]
suspect_user = list(check_user['userid'])
suspect_user

[65129646, 86829555]

In [21]:
ans_dic= {}

for i in data.index:
  initial = data['date_time'].loc[i]
  for j in range(i, data.last_valid_index()+1):
    delta = data['date_time'].loc[j] - initial
    if(delta.total_seconds()<3600):
      check = data.loc[i:j].groupby('shopid')
      key = list(check.groups.keys())
      for k in key:
        check_order = check.get_group(k)
        user_num = pd.DataFrame(check_order.groupby('userid')['userid'].count().reset_index(name='user_count'))
        check_user = user_num[user_num['user_count'] >= 3]
        suspect_user = list(check_user['userid'])
        order = len(check_order.orderid)
        buyers = len(check_order.userid.unique())
        concentrate_rate = order/buyers
        if(concentrate_rate >= 3):
          if k in ans_dic:
            ans_dic[k].append(suspect_user)
          else:
            ans_dic[k] = [suspect_user]

In [22]:
ans_dic

{68986766: [[65129646, 86829555],
  [65129646, 86829555],
  [65129646, 86829555],
  [65129646, 86829555],
  [65129646, 86829555],
  [65129646, 86829555],
  [65129646, 86829555],
  [65129646, 86829555],
  [65129646, 86829555],
  [65129646, 86829555],
  [86829555],
  [86829555],
  [86829555],
  [86829555],
  [86829555],
  [65129646, 86829555],
  [65129646],
  [65129646],
  [65129646],
  [65129646],
  [65129646]],
 85669731: [[112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421],
  [112494421]]}

In [23]:
shop_list = []
user_list = []

for key, value in ans_dic.items():
  shop_list.append(key)
  user_list.append(value)

In [24]:
column_names = ["shopid", "user_list"]
df = pd.DataFrame(columns = column_names)
df['shopid'] = shop_list
df['user_list'] = user_list
df

,shopid,user_list
0,85669731,"[[112494421], [112494421], [112494421], [11249..."
1,68986766,"[[65129646, 86829555], [65129646, 86829555], [..."


In [28]:
for n in range(0, df.last_valid_index()+1):
  res = []
  for i in df['user_list'].loc[n]:
    for j in i:
      if j not in res:
        res.append(j)
  df['userid'].loc[n] = res

KeyError: ignored

In [70]:
ans_table = df.drop(['user_list'], axis=1)
ans_table

,shopid,userid
0,85669731,[112494421]
1,68986766,"[65129646, 86829555]"


In [72]:
ans_table['unique_id'] = ['&'.join(map(str, l)) for l in ans_table['userid']]
final = ans_table.drop([''])

,shopid,userid,unique_id
0,85669731,[112494421],112494421
1,68986766,"[65129646, 86829555]",65129646&86829555


In [68]:
removed_userlist

,shopid,userid,unique_id
0,85669731,[112494421],112494421
1,68986766,"[65129646, 86829555]",65129646&86829555


In [ ]:
df.to_csv('CY_Team_submission4.csv') 
files.download('CY_Team_submission4.csv')